In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

import yaml

import pseudopeople as pp
from pseudopeople.utilities import get_configuration
from vivarium.framework.randomness import RandomnessStream

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import datatypes
from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Tue 04 Apr 2023 04:06:15 PM PDT
ndbs
Linux int-slurm-sarchive-p0003 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


In [3]:
18*45/60

13.5

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
819/554

1.4783393501805053

# Find data

```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/full_scale_334mil/united_states_of_america/2023_03_30_10_23_13/final_results/2023_03_30_16_02_39
```

In [6]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/full_scale_334mil'
    '/united_states_of_america/2023_03_30_10_23_13'
    '/final_results/2023_03_30_16_02_39'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 404K
drwxrwsr-x  3 sbachmei IHME-Simulationscience 1.5K Mar 30 17:15 ..
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:52 tax_dependents_observer
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:44 tax_1040_observer
drwxrwsr-x  2 sbachmei IHME-Simulationscience 339K Mar 30 16:42 logs
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:31 tax_w2_observer
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:28 social_security_observer
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:28 wic_observer
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:28 household_survey_observer_cps
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:28 household_survey_observer_acs
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Mar 30 16:26 decennial_census_observer
drwxrwsr-x 11 sbachmei IHME-Simulationscience 4.5K Mar 30 16:21 .


In [7]:
!ls -halt $output_dir/tax_w2_observer

total 199G
-rw-r--r--  1 sbachmei IHME-Simulationscience 614M Mar 30 17:07 tax_w2_observer_3568.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 603M Mar 30 17:04 tax_w2_observer_7551.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 613M Mar 30 17:04 tax_w2_observer_7086.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 613M Mar 30 17:00 tax_w2_observer_5440.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 598M Mar 30 16:58 tax_w2_observer_9292.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 606M Mar 30 16:42 tax_w2_observer_1282.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 611M Mar 30 16:39 tax_w2_observer_1007.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 610M Mar 30 16:34 tax_w2_observer_2277.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 606M Mar 30 16:34 tax_w2_observer_1483.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 597M Mar 30 16:33 tax_w2_observer_1482.csv.bz2
-rw-r--r--  1 sbachmei IHME-Simulationscience 606M Mar 30 1

# Load keyspace.yaml to get a list of keys so we can choose a few

In [8]:
!ls $project_output_dir/results/full_scale_334mil/united_states_of_america/2023_03_30_10_23_13

branches.yaml  logs			 __pycache__	   settings.py
final_results  model_specification.yaml  raw_results
keyspace.yaml  output.hdf		 requirements.txt


In [16]:
keyspace_path = (
    f'{project_output_dir}/results'
    '/full_scale_334mil/united_states_of_america/2023_03_30_10_23_13'
    '/keyspace.yaml'
)
with open(keyspace_path) as stream:
    keyspace = yaml.safe_load(stream)
    
print(type(keyspace))
keyspace.keys()

<class 'dict'>


dict_keys(['input_draw', 'placeholder_branch_name.scenario', 'random_seed'])

In [20]:
print("Number of seeds:", len(keyspace['random_seed']))
keyspace['random_seed'][:5]

Number of seeds: 334


[4344, 5616, 6810, 2787, 2284]

In [23]:
# Choose 3 seeds from some starting position i
i = 4
seeds = keyspace['random_seed'][i:i+3]
seeds

[2284, 4369, 3254]

In [25]:
min(keyspace['random_seed']), max(keyspace['random_seed'])

(28, 9971)

In [28]:
np.log10(20)

1.3010299956639813

In [56]:
np.log10(10)

1.0

In [33]:
np.log10(9971)

3.9987387162558177

In [57]:
np.log10(1_000)

3.0

In [58]:
np.log10(9_999)

3.9999565683801928

In [59]:
seed_lengths = pd.Series(1+np.floor(np.log10(seed)) for seed in keyspace['random_seed'])
seed_lengths.value_counts()

4.0    303
3.0     27
2.0      4
dtype: int64

# Do some runtime, storage, and memory usage calculations

In [35]:
4.34*18

78.12

In [38]:
15449588*1e9/2**30

14388550.072908401

In [37]:
15/24

0.625

In [39]:
15*340/18

283.3333333333333

In [40]:
.625*450

281.25

In [41]:
38526.828731999994*340/18

727728.9871599999

In [43]:
350*18/1000

6.3

In [44]:
.12*60

7.199999999999999

In [46]:
45*18/60

13.5

In [48]:
38526.828731999994 * 1e9 / 2**30

35880.905326455824

In [49]:
16 * 1e9 / 2**30

14.901161193847656

In [50]:
54 * 1e6/2**20

51.4984130859375

In [51]:
23*334

7682

In [52]:
7682*1e9/2**30

7154.420018196106

In [53]:
7682*2**20/1e6 * 1e9/2**30

7501.953125

In [54]:
7300/983

7.426246185147508

In [55]:
983/7300

0.13465753424657534

In [60]:
22.3*20/60

7.433333333333334